# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# Class WSM

In [2]:
class WSM:
    def __init__(self, data, encoding_column, encoder): #constructor
        self.data = data #input data
        self.encoding_column = encoding_column
        self.encoder = encoder
        self.beneficiary_or_no = None
        self.weights = None
        
    def find_weights_and_beneficiary(self):
        #find weights and beneficiary from dataset
        self.beneficiary_or_no = self.data.iloc[0,:].values
        self.weights = self.data.iloc[1,:].values
        self.data = self.data.iloc[2:,:] #remove first two rows
        print('Beneficiary :', self.beneficiary_or_no)
        print('Weights :', self.weights)
        display(self.data)
        
    def column_encoder(self):
        #encode categorical columns
        self.data = self.data.replace({self.encoding_column : self.encoder})
        print("After Encoding Column : ")
        display(self.data)
        
    def min_max_normalization(self):
        #min max normalization
        for i in range(1,len(self.beneficiary_or_no)):
            if(self.beneficiary_or_no[i] == 'B'):
                max_val = np.max(self.data.iloc[:,i])
                self.data.iloc[:,i] /= max_val
            else:
                min_val = np.min(self.data.iloc[:,i])
                self.data.iloc[:,i] = min_val / self.data.iloc[:,i]
        print("After Min Max Normalization : ")
        display(self.data)
                
    def apply_weights(self):
        #apply weights
        self.data.iloc[:,1:] = self.data.iloc[:,1:].mul(np.array(self.weights[1:]))
        print("After applying Weights : ")
        display(self.data)
        
    def calc_selection_index_rank(self):
        #calculate Selection index
        self.data.reset_index(drop=True, inplace=True)
        self.data['SI'] = np.sum(self.data.iloc[:,1:], axis=1)
        self.data['Rank'] = self.data['SI'].rank(axis=0, ascending=False)
        print("After Calculating Selection Index & Rank: ")
        display(self.data)
        
    def run_all(self):
        #run all steps
        self.find_weights_and_beneficiary()
        self.column_encoder()
        self.min_max_normalization()
        self.apply_weights()
        self.calc_selection_index_rank()

# Import Data

In [3]:
data = pd.read_excel('WSM_1.xlsx')
#data = pd.read_excel('WSM_2.xlsx')
data

,Alternative / Criteria,Cost,CC,Mileage,Performance
0,NaN,NB,B,B,B
1,NaN,0.25,0.25,0.25,0.25
2,Yamaha,150000,250,35,Very Good
3,RE,180000,350,30,Good
4,Bajaj,125000,220,40,Good
5,Honda,90000,150,48,Better


# Create Object and Run All

In [4]:
encoding_column = "Performance"
performance_encoder = {'Excellent':5, 'Very Good':4, 'Good':3, 'Better':2, 'Worse':1}
wsm_object = WSM(data, encoding_column, performance_encoder)

In [5]:
wsm_object.run_all()

Beneficiary : [nan 'NB' 'B' 'B' 'B']
Weights : [nan 0.25 0.25 0.25 0.25]


,Alternative / Criteria,Cost,CC,Mileage,Performance
2,Yamaha,150000,250,35,Very Good
3,RE,180000,350,30,Good
4,Bajaj,125000,220,40,Good
5,Honda,90000,150,48,Better


After Encoding Column : 


,Alternative / Criteria,Cost,CC,Mileage,Performance
2,Yamaha,150000,250,35,4
3,RE,180000,350,30,3
4,Bajaj,125000,220,40,3
5,Honda,90000,150,48,2


After Min Max Normalization : 


,Alternative / Criteria,Cost,CC,Mileage,Performance
2,Yamaha,0.6,0.714286,0.729167,1.00
3,RE,0.5,1.0,0.625,0.75
4,Bajaj,0.72,0.628571,0.833333,0.75
5,Honda,1.0,0.428571,1.0,0.50


After applying Weights : 


,Alternative / Criteria,Cost,CC,Mileage,Performance
2,Yamaha,0.15,0.178571,0.182292,0.25
3,RE,0.125,0.25,0.15625,0.1875
4,Bajaj,0.18,0.157143,0.208333,0.1875
5,Honda,0.25,0.107143,0.25,0.125


After Calculating Selection Index & Rank: 


,Alternative / Criteria,Cost,CC,Mileage,Performance,SI,Rank
0,Yamaha,0.15,0.178571,0.182292,0.25,0.760863,1.0
1,RE,0.125,0.25,0.15625,0.1875,0.718750,4.0
2,Bajaj,0.18,0.157143,0.208333,0.1875,0.732976,2.0
3,Honda,0.25,0.107143,0.25,0.125,0.732143,3.0
